# The procedure of encription and decryption is shown in the following:

In [ ]:
import numpy as np

In [ ]:
n = 8

In [ ]:
L = n * np.identity(n)

## We generate a random unitary matrice

To generate an unitary matrice, we multiply each row of an n×n identity matrix with -1 or 1 uniformly at random. Then we permute the rows (or eqvalently columns) randomly uniformly. We break out of the loop if the determinant of the matrice is 1.

In [ ]:
# generate random U
while True:
    rnd = (np.random.choice([-1,1], n) * np.identity(n)).astype(int)
    random_permut = np.random.permutation(n)
    U = rnd[random_permut]
    if np.isclose(np.linalg.det(U), 1.0): break
print(U, np.linalg.det(U))

[[ 0  0  0 -1  0  0  0  0]
 [ 0  0  0  0  0  0  0 -1]
 [ 1  0  0  0  0  0  0  0]
 [ 0 -1  0  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  0 -1  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  1  0  0]] 1.0


## Here we generate a rigid rotation matrice with rational entries


In [ ]:
# generate a set of linearly independent vectors and a rigid rotation matrix
while True:
    vectors = np.random.randint(-10, 10, n*n).reshape((n,n))
    if np.linalg.matrix_rank(vectors) == n: break
orthbasis, uptri = np.linalg.qr(vectors)
R = orthbasis
print(R, np.linalg.det(R))

[[-0.6704784  -0.27277411 -0.36750124  0.11355483  0.2557279  -0.020284
   0.35615981  0.36802599]
 [ 0.09578263 -0.01236136 -0.00517148  0.60976965 -0.64741554  0.09628139
   0.43633216  0.0049364 ]
 [ 0.09578263  0.43676821 -0.48089872 -0.08377247 -0.28274944  0.12289797
  -0.35056786  0.5863692 ]
 [ 0.         -0.71860732  0.27852763 -0.2109783  -0.36457461  0.12374325
  -0.29449123  0.35575541]
 [ 0.09578263 -0.19201319 -0.5388345  -0.5497078  -0.348845   -0.19681438
   0.2652722  -0.36143778]
 [-0.38313051 -0.04038046 -0.12548659  0.31929741 -0.21416985 -0.53820893
  -0.56047579 -0.29019307]
 [-0.38313051  0.04944546 -0.09997306 -0.02555134 -0.11816962  0.77852142
  -0.22242517 -0.41302064]
 [-0.47891314  0.42111048  0.48988852 -0.39871573 -0.34794948 -0.15937939
   0.19790433  0.09330815]] -1.0


## Compute $\hat{L}$

In [ ]:
L_hat = np.matmul(np.matmul(U,L), R)

# look all columns and rows of L_hat are of same length
# and also look that L_hat * L_hat.T = n**2 I
print('cols-norm:', [np.linalg.norm(row) for row in L_hat.transpose()])
print('rows-norm:',[np.linalg.norm(row) for row in L_hat])
print(f'self*transpose:\n{np.matmul(L_hat, L_hat.transpose()).round()}')

cols-norm: [8.000000000000002, 7.999999999999999, 8.0, 8.0, 7.999999999999999, 8.0, 7.999999999999999, 8.0]
rows-norm: [8.0, 8.0, 8.0, 7.999999999999999, 8.0, 8.0, 8.0, 8.0]
self*transpose:
[[64.  0.  0. -0.  0.  0.  0.  0.]
 [ 0. 64.  0. -0. -0. -0. -0. -0.]
 [ 0.  0. 64.  0.  0. -0.  0. -0.]
 [-0. -0.  0. 64. -0.  0. -0.  0.]
 [ 0. -0.  0. -0. 64.  0. -0. -0.]
 [ 0. -0. -0.  0.  0. 64.  0. -0.]
 [ 0. -0.  0. -0. -0.  0. 64.  0.]
 [ 0. -0. -0.  0. -0. -0.  0. 64.]]


# Encryption stage

In [ ]:
# encription
v = np.random.randint(-100,100,n).reshape((1,n))
m = np.random.randint(0,2,n).reshape((1,n))
c = np.matmul(v,L_hat) + m
print(c, m)

[[  259.2299665   -577.03058491  -270.9405042    -10.49715089
    326.16858599  -582.8450789  -1062.11341971    66.50019824]] [[1 1 1 1 1 1 1 0]]


# Decryption stage

In [ ]:
# decryption
d = np.matmul(c, R.transpose())
print(d)
for j in range(n):
        d[0,j] = d[0,j]%n
        if d[0,j] > n/2: d[0,j] -= n
        elif d[0,j] < -n/2: d[0,j] += n
print(d)

md = np.matmul(d, R)
print(md)
print(md.round().astype(int))
assert (md.round().astype(int) == m).all()

[[-176.60559521 -703.42678257  151.45746032  486.81361942  -17.46516004
   774.45744528 -384.02128283 -720.27605442]]
[[-0.60559521  0.57321743 -0.54253968 -1.18638058 -1.46516004 -1.54255472
  -0.02128283 -0.27605442]]
[[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 7.90478794e-14]]
[[1 1 1 1 1 1 1 0]]


# Attack on system

We don't even need to find the basis of the original private key $R$. 

In [ ]:
# check correct decription for a different R for a different U_prime
U_prime = np.identity(n)
R_prime = L_hat/n

# decryption
d_prime = np.matmul(c, R_prime.transpose())
print(d_prime)
for j in range(n):
        d_prime[0,j] = d_prime[0,j]%n
        if d_prime[0,j] > n/2: d_prime[0,j] -= n
        elif d_prime[0,j] < -n/2: d_prime[0,j] += n
print(d_prime)

md_prime = np.matmul(d_prime, R_prime)
print(md_prime)
print(md_prime.round().astype(int))
assert (md_prime.round().astype(int) == m).all()

[[-486.81361942  720.27605442 -176.60559521  703.42678257   17.46516004
  -151.45746032 -384.02128283  774.45744528]]
[[ 1.18638058  0.27605442 -0.60559521 -0.57321743  1.46516004  0.54253968
  -0.02128283 -1.54255472]]
[[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 1.00000000e+00 7.90478794e-14]]
[[1 1 1 1 1 1 1 0]]


# Checking the success of the attack for randomly generated parameters R, U, and m

If the attack is unsuccesfull for some choosen parameters, then there will be printed an assertion error.

In [ ]:
# check correct decription for a different R for a different U_prime for random instances of m and U
n=10
L = n * np.identity(n)
for repete in range(1000):
    # generate random U
    while True:
        rnd = (np.random.choice([-1,1], n) * np.identity(n)).astype(int)
        random_permut = np.random.permutation(n)
        U = rnd[random_permut]
        if np.isclose(np.linalg.det(U), 1.0): break
    # print(U, np.linalg.det(U))
    # generate a set of linearly independent vectors and a rigid rotation matrix
    while True:
        vectors = np.random.randint(-1000, 1000, n*n).reshape((n,n))
        if np.linalg.matrix_rank(vectors) == n: break
    orthbasis, uptri = np.linalg.qr(vectors)
    R = orthbasis
    # print(R, np.linalg.det(R))
    L_hat = np.matmul(np.matmul(U,L), R)
    
    # encription
    v = np.random.randint(-100,100,n).reshape((1,n))
    m = np.random.randint(0,2,n).reshape((1,n))
    c = np.matmul(v,L_hat) + m
    # print(c, m)

    # check correct decription for a different R for a different U_prime
    U_prime = np.identity(n)
    R_prime = L_hat/n

    # decryption
    d_prime = np.matmul(c, R_prime.transpose())
    # print(d_prime)
    for j in range(n):
            d_prime[0,j] = d_prime[0,j]%n
            if d_prime[0,j] > n/2: d_prime[0,j] -= n
            elif d_prime[0,j] < -n/2: d_prime[0,j] += n
    # print(d_prime)

    md_prime = np.matmul(d_prime, R_prime)
    assert (md_prime.round().astype(int) == m).all()



# Checking the decryption of the cipher for randomly generated parameters R, U, and m


In [ ]:
# check if correct decription for some n for all possible m
n=3

# U = np.identity(n)
L = n * np.identity(n)

for repeats in range(10):
    # R = np.array([[0.6, -0.8], [0.8, 0.6]])

    # generate a set of linearly independent vectors and a rigid rotation matrix
    while True:
        vectors = np.random.randint(-1000, 1000, n*n).reshape((n,n));
        if np.linalg.matrix_rank(vectors) == n: break
    orthbasis, uptri = np.linalg.qr(vectors)
    R = orthbasis

    # assert np.isclose(abs(np.linalg.det(R)), 1)
    # assert np.isclose(np.matmul(R, R.transpose()), np.identity(n)).all()

    # generate random U
    while True:
        rnd = (np.random.choice([-1,1], n) * np.identity(n)).astype(int)
        random_permut = np.random.permutation(n)
        U = rnd[random_permut]
        if np.isclose(np.linalg.det(U), 1.0): break
    # print(U, np.linalg.det(U))

    L_hat = np.matmul(np.matmul(U,L), R)
    v = np.random.randint(-100,100,n).reshape((1,n))


    for m in range(2**n):
        # encryption
        m = np.array([*bin(m)[2:].zfill(n)], dtype=float).reshape((1,n))
        c = np.matmul(v,L_hat) + m

        # decryption
        d = np.matmul(c, R.transpose()) 
        for j in range(n):
            d[0,j] = d[0,j]%n
            if d[0,j] > n/2: d[0,j] -= n
            elif d[0,j] < -n/2: d[0,j] += n
        
        md = np.matmul(d, R)

        if np.isclose(md, m).all(): continue
        print(m,md)
        print(U)
        print(R)
        print(v, '\n')

[[1. 1. 1.]] [[-0.83115651 -0.88636407 -0.44516312]]
[[ 0 -1  0]
 [-1  0  0]
 [ 0  0 -1]]
[[-0.6103855  -0.62878802 -0.48172104]
 [ 0.52607811 -0.77645948  0.34691856]
 [-0.59217511 -0.04166883  0.80473123]]
[[ 94  50 -91]] 

[[1. 1. 1.]] [[-0.36145685 -1.57960685  0.29852747]]
[[0 1 0]
 [0 0 1]
 [1 0 0]]
[[-0.45381895 -0.85986895 -0.23382418]
 [ 0.2551628  -0.37680988  0.89045284]
 [-0.85378001  0.34444114  0.39041003]]
[[ 54  24 -41]] 



## Verify that every cipher text $c = v\hat{L}+m$ maps to an unique $m$

If this was not the case the code will output more than once. The first output is trivial for $m_1 = m_2$ and $v_1 = v_2$.

The idea is that if $(m_1 - m_2)\times\hat{L}^{-1}=v_1 - v_2$ is an integer then we have the cipher $c_1=v_1\times\hat{L}+m_1 = c_2=v_2\times\hat{L}+m_2$.We bruteforce over all possible values of $m_1-m_2$ for a randomly choosen U, R.

In [ ]:
# 0 = (v1-v2)*L_hat + (m1 - m2)
# (m1-m2)*L_hat_inv = v1-v2

import numpy as np

n = 10

L = n * np.identity(n)

# generate random U
while True:
    rnd = (np.random.choice([-1,1], n) * np.identity(n)).astype(int)
    random_permut = np.random.permutation(n)
    U = rnd[random_permut]
    if np.isclose(np.linalg.det(U), 1.0): break

# generate a set of linearly independent vectors and a rigid rotation matrix
while True:
    vectors = np.random.randint(-10, 10, n*n).reshape((n,n))
    if np.linalg.matrix_rank(vectors) == n: break
orthbasis, uptri = np.linalg.qr(vectors)
R = orthbasis

# make the L_hat
L_hat = np.matmul(np.matmul(U,L), R)

# the bruteforce part
M = [-1, 0, 1]
L_hat_inv = np.linalg.inv(L_hat)
for i in range(3**n):
    m1mm2 = [-1]*n; k = 0 # m1mm2 is m1 - m2 -> each element can take value in [-1, 0, 1]
    while i > 0:
        m1mm2[n-k-1] = M[i%3]
        i = i//3
        k += 1
    m1mm2 = np.array([m1mm2])
    # if this is an integer then m1 and m2 can correspnond to same cipher 
    v1mv2 = np.matmul(m1mm2, L_hat_inv)
    if np.allclose(v1mv2, v1mv2.round(), rtol=1e-10):
        print(m1mm2, v1mv2.round())

# An Attack of the bruteforce way
we bruteforce over $m$ to find a $v$ with integer entries

In [37]:
n = 10
m = np.random.randint(0,2,n).reshape((1,n)) # a random m
print('actual message', m)

L = n * np.identity(n)
# generate random U
while True:
    rnd = (np.random.choice([-1,1], n) * np.identity(n)).astype(int)
    random_permut = np.random.permutation(n)
    U = rnd[random_permut]
    if np.isclose(np.linalg.det(U), 1.0): break
# generate a set of linearly independent vectors and a rigid rotation matrix
while True:
    vectors = np.random.randint(-10, 10, n*n).reshape((n,n))
    if np.linalg.matrix_rank(vectors) == n: break
orthbasis, uptri = np.linalg.qr(vectors)
R = orthbasis
# make the L_hat
L_hat = np.matmul(np.matmul(U,L), R)

# get a cipher
v = np.random.randint(-100,100,n).reshape((1,n))
c = np.matmul(v,L_hat) + m
print('cipher', c)

# the bruteforce part:
L_hat_inv = np.linalg.inv(L_hat)
for m_guess in range(2**n):
    m_guess = np.array([*bin(m_guess)[2:].zfill(n)], dtype=float).reshape((1,n))
    v_guess = np.matmul((c-m_guess), L_hat_inv)
    if np.allclose(v_guess, v_guess.round(), rtol=1e-10):
        print('guessed m:', m_guess.round().astype(int))
        break


actual message [[1 1 1 0 0 0 1 1 1 1]]
cipher [[ 160.90053727  889.22884321 -565.18535851  862.06847579  -58.34625953
   143.78993871  858.01392199 -136.54326116 -415.77070205 1096.43815955]]
guessed m: [[1 1 1 0 0 0 1 1 1 1]]


## Code used for testing purposes and discarded code - just there for completeness

In [ ]:
# # checking orthogonality of L_hat for random U
# for n in [50]:

#     n2I = n**2 * np.identity(n)
#     L = n * np.identity(n)
#     for repeats in range(100):

#         # generate random U
#         while True:
#             rnd = (np.random.choice([-1,1], n) * np.identity(n)).astype(int)
#             random_permut = np.random.permutation(n)
#             U = rnd[random_permut]
#             if np.isclose(np.linalg.det(U), 1.0): break
            
#         # generate a set of linearly independent vectors and a rigid rotation matrix
#         while True:
#             vectors = np.random.randint(-1000, 1000, n*n).reshape((n,n))
#             if np.linalg.matrix_rank(vectors) == n: break
#         orthbasis, uptri = np.linalg.qr(vectors)
#         R = orthbasis

#         L_hat = np.matmul(np.matmul(U,L), R)
#         assert np.allclose([np.linalg.norm(row) for row in L_hat.transpose()], n)
#         assert np.allclose([np.linalg.norm(row) for row in L_hat], n)
#         assert np.allclose(np.matmul(L_hat, L_hat.transpose()), n2I)
    

In [ ]:
# # generate random U by integer eigen values
# I = np.identity(n)
# while True:
#     Eigens = np.random.randint(-1, 2, n) * np.identity(n, dtype=int)
#     # print(Eigens)

#     while True:
#         V = np.random.randint(-10, 10, n*n).reshape((n,n))
#         det_V = np.linalg.det(V)
#         if not np.isclose(0.0, det_V, rtol=1e-10): break
        
#     # U is an integer matrix
#     U = det_V * np.matmul(np.matmul(V, Eigens), np.linalg.inv(V))
#     det_U = np.linalg.det(U)

#     if np.isclose(det_U, 0.0, rtol=1e-10): continue

#     U = U / (det_U ** (1/n))

#     det_U = np.linalg.det(U)
#     if np.allclose(U, I, rtol=1e-10): continue
#     if not np.isclose(det_U, 1.0, rtol=1e-10): continue
#     if not np.allclose(U, np.round(U), rtol=1e-10): continue
#     if np.allclose(np.matmul(U, U.transpose()), I): break # final condition for unitary

# print('before round: ', U, 'det: ', np.linalg.det(U))

# U = np.round(U)

# print(U)

In [ ]:
# def gram_schmidt(A):
#     """Orthogonalize a set of vectors stored as the columns of matrix A."""
#     # Get the number of vectors.
#     n = A.shape[1]
#     for j in range(n):
#         # To orthogonalize the vector in column j with respect to the
#         # previous vectors, subtract from it its projection onto
#         # each of the previous vectors.
#         for k in range(j):
#             A[:, j] -= np.dot(A[:, k], A[:, j]) * A[:, k]
#         A[:, j] = A[:, j] / np.linalg.norm(A[:, j])
#     return A

# if __name__ == '__main__':
#     print(gram_schmidt(vectors.transpose()))

In [ ]:
# def gram_schmidt(vectors):
#     basis = []
#     for v in vectors:
#         w = v - np.sum( [np.dot(v,b)*b  for b in basis] )
#         if (abs(w) > 1e-10).any():  
#             basis.append(w/np.linalg.norm(w))
#     return np.array(basis)

# orthbasis1 = gram_schmidt(vectors)
# np.matmul(orthbasis1, orthbasis1.transpose())

In [ ]:
# A = vectors.copy()

# def normalize(v):
#     return v / np.sqrt(v.dot(v))

# n = len(A)

# A[:, 0] = normalize(A[:, 0])

# for i in range(1, n):
#     Ai = A[:, i]
#     for j in range(0, i):
#         Aj = A[:, j]
#         t = Ai.dot(Aj)
#         Ai = Ai - t * Aj
#     A[:, i] = normalize(Ai)

# print(A)

In [ ]:
# import numpy as np


# def gram_schmidt(A):
    
#     (n, m) = A.shape
#     print(A)
    
#     for i in range(m):
        
#         q = A[:, i] # i-th column of A
        
#         for j in range(i):
#             q = q - np.dot(A[:, j], A[:, i]) * A[:, j]
        
#         if np.array_equal(q, np.zeros(q.shape)):
#             raise np.linalg.LinAlgError("The column vectors are not linearly independent")
        
#         # normalize q
#         q = q / np.sqrt(np.dot(q, q))
        
#         # write the vector back in the matrix
#         A[:, i] = q


# C = vectors.copy()
# gram_schmidt(C)
# print(C)

In [ ]:
# n = 6
# I = np.identity(n)

# # make a rational orthogonal matrix
# S = np.zeros((n,n))
# for i in range(n):
#     for j in range(n): 
#         if j == i: continue
#         S[i,j] = np.random.randint(-100,100)
#         S[j,i] = -S[i,j] 
# R = np.linalg.inv(S-I)*(S+I)
# print('S:',S)
# print('R:',R)
# print('det:', np.linalg.det(R))
# R /= (np.linalg.det(R))**(1/n)